In [ ]:
#Hugging face에 있는 lcw99/t5-base-korean-text-summary (한글로 된 장문을 짧게 요약해주는 모델) 모델을 이용하여 뉴스 기사 요약 및 키워드 추출
#개선 방안->요약 내용이 좀 더 길면 좋을 것 같음

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install transformers
!pip install torch
!pip install konlpy
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 36.3 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import re


model_name = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 요약 파이프라인 설정
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 네이버 API 키 설정
client_id = 'xIpH54vcwURuvzz8t0vl'
client_secret = 'dfHi2SP3CQ'

# API 요청 URL
url = 'https://openapi.naver.com/v1/search/news.json'

# 카테고리 설정
categories = {
    '경제': '경제',
    'IT': 'IT',
    '과학': '과학',
    '사회': '사회',
    '문화': '문화',
    '스포츠': '스포츠',
    '건강': '건강'
}

# 사용자로부터 카테고리 선택 받기
print("카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강")
user_choice = input("선택한 카테고리: ")

# 유효한 카테고리인지 확인
if user_choice in categories:
    selected_category = categories[user_choice]
else:
    print("유효하지 않은 카테고리입니다. 기본적으로 '경제' 카테고리를 선택합니다.")
    selected_category = '경제'

# 파라미터 설정
params = {
    'query': selected_category,  # 검색어
    'display': 5,            # 5개의 뉴스 가져오기
    'sort': 'date'           # 날짜 기준 정렬
}

# 헤더 설정
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# 기사 내용 추출 함수
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve the web page. Status code: {response.status_code}")
            return ""

        soup = BeautifulSoup(response.content, 'html.parser')
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()

        text = soup.get_text(separator=' ')
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Error during requests to {url}: {str(e)}")
        return ""

# API 요청
response = requests.get(url, params=params, headers=headers)

# 응답을 JSON 형태로 파싱
data = response.json()

# 뉴스 정보 출력 및 기사 내용 추출
if 'items' in data:
    articles = data['items']
    for i, article in enumerate(articles, 1):
        title = article['title']
        description = article['description']
        link = article['link']
        pub_date = article['pubDate']

        print(f"뉴스 {i}:")
        print(f"제목: {title}")
        print(f"URL: {link}")
        print(f"발행일: {pub_date}")

        # URL에서 기사 텍스트 추출
        extracted_text = extract_text_from_url(link)

        if extracted_text:
            # KoBART 요약 기능
            summary = summarizer(extracted_text, max_length=300, min_length=100, do_sample=False)
            summary_text = summary[0]['summary_text']

            # 요약된 텍스트가 3000자를 넘지 않도록 자르기
            if len(summary_text) > 3000:
                summary_text = summary_text[:3000] + '...'

            print("내용 요약:")
            print(summary_text)

            # 키워드 추출 기능
            okt = Okt()

            def extract_keywords(text, num_keywords=5):
                text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)
                nouns = okt.nouns(text)
                if not nouns:
                    return []
                vectorizer = TfidfVectorizer(max_features=num_keywords)
                tfidf_matrix = vectorizer.fit_transform([' '.join(nouns)])
                feature_names = vectorizer.get_feature_names_out()
                sorted_items = sorted(zip(vectorizer.idf_, feature_names))
                keywords = [item[1] for item in sorted_items[:num_keywords]]
                return keywords

            keywords = extract_keywords(summary_text, num_keywords=5)

            print(f"키워드: {keywords}")
        else:
            print("기사를 추출할 수 없습니다.")
        print("-" * 80)
else:
    print("뉴스를 가져오는데 실패했습니다.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강
선택한 카테고리: 경제
뉴스 1:
제목: 강북구 평생학습 '생활 속 다산 사상' 신청하세요
URL: http://www.m-i.kr/news/articleView.html?idxno=1154159
발행일: Wed, 28 Aug 2024 14:56:00 +0900
내용 요약:
이 대표는 코로나19 이후 사회적 거리두기 등으로 인해 사회적 거리두기 완화 등 사회적 거리두기 완화를 위해 노력하고 있다고 말했다. 이 기자는 서울 강북구 평생학습 프로그램 생활 속 다산 사상에 참여할 수강생을 모집한다고 밝혔다.                           
키워드: ['강북구', '거리', '두기', '사회', '완화']
--------------------------------------------------------------------------------
뉴스 2:
제목: 세종뮤지엄갤러리, 폴 정 기획초대전 개최
URL: https://www.kfenews.co.kr/news/articleView.html?idxno=624816
발행일: Wed, 28 Aug 2024 14:56:00 +0900
내용 요약:
세종뮤지엄갤러리에서 열리는 전시에는 현대미술관 외 20개 공공기관에 작품을 소장하고 있는 폴 정 작가의 작품이 전시된다.                                                                     
키워드: ['갤러리', '공공기관', '미술관', '작품', '전시']
--------------------------------------------------------------------------------
뉴스 3:
제목: 임신준비 부부, 8월 30일부터 '냉동난자 사용 보조생식술' 지원 온라인 ...
URL: https://www.foodneconomy.com/news/articleView.html?idxno=361947
발행일: 